# Omani-Voice-Therapist

## Tools Intailization, Libraries importing, LLMs assignmed

### General Intializations


#### Libraries Importing

In [18]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, AnyMessage
from langchain_core.runnables.base import RunnableSerializable
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableField
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from openai import OpenAI

from sentence_transformers import SentenceTransformer, util
import azure.cognitiveservices.speech as speechsdk
from pydantic import BaseModel, Field

import os
import numpy as np
import asyncio
import logging
import io
import uuid
import soundfile as sf
import gradio as gr

##### Intializing LLMs and Framework

In [19]:
## Intializng OpenAI
openai_model = "gpt-4o"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
GPT4o = ChatOpenAI(temperature = 0, model= openai_model, streaming = True).configurable_fields(
    callbacks=ConfigurableField(
        id="callbacks",
        name="callbacks",
        description="A list of callbacks to use for streaming",
    )
)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Intailzing Claude
claude_model = "claude-3-7-sonnet-latest"
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
ClaudeSonnet3_7 = ChatAnthropic(temperature= 0, model_name= claude_model, streaming= True).configurable_fields(
    callbacks=ConfigurableField(
        id="callbacks",
        name="callbacks",
        description="A list of callbacks to use for streaming",
    )
)

## Intailzing Langchain 
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "omani-therapist-voice"

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

## Azure Speech Services API Key
SPEECH_KEY =  os.getenv("SPEECH_KEY")
SPEECH_ENDPOINT = os.getenv("ENDPOINT")

#### Logger Intialization and Creation

In [20]:
# 🧭 Logger Setup
def setup_logging():
    """Configure logging for the application"""
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s | %(levelname)s | %(message)s",
        handlers=[logging.StreamHandler()]
    )
    return logging.getLogger(__name__)


logger = setup_logging()

## Agent Creation

##### General Tools

In [21]:
tavily_search = TavilySearch(
    max_results=5,
    topic="general",
)

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

# Define the multiply tool
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

# Define the exponentiate tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

def Notify(email):
    """Notify Authorities if patient emotional Health is in critical conditions or he/she are mentioning about harming themselves or others"""
    return "Notified Police and Called an Ambulance, and " + email

@tool
def health_searcher(query : str):
    """Use this function strictly when the user asks for you to search about mental health therapy techniques, nearest mental hospitals, 
    mental health medications in Oman, hotlines and nothing else"""
    return tavily_search.invoke(query)

@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

tools = [add, multiply, exponentiate, subtract, final_answer, health_searcher]
name2tool = {tool.name : tool.func for tool in tools}

##### Adjusting System Prompts

In [22]:
SYSTEM_PROMPT_RISK_ANALYZER = \
(
    "You are an Omani AI Therapist, and you're task is to provide solutions for the users emotional issues. \n"
    "Also, You're task is to analyze the user's prompt emotion/feeling.\n"
    "You will try to provide solutions for users and help them using Cogonitive Behaviour Techniques (CBT), if it is possible, otherwise put None.\n"
    "You will mention the reason behind your selection for this, and how to apply it .\n"
    "You will try to include, if posssible, omani and islamic values for the cultural context for how to solve this problem.\n"
    "You will fill 8 variables: requires_analysis, user_emotion, severity, diagnosis, user_prompt, cbt_technique, reason_for_technique, cultural_context \n"
    "If you did not manage to fill any of those variables just put None, and do not add anything else"
    "You will reply in arabic and be brief as possible"
)

SYSTEM_PROMPT_THERAPIST = \
(
    "You are an Omani AI Therapist, and you will recieve a structured enhanced prompt of patients.\n"
    "Your task is reply back to the user in authentic omani arabic"
    "If user's problem is not mentioned, you can encourge him/her to give more details about his/her problem"
    "You must adhere to omani gulf culture, and Islamic context, and not mention anything that contradicts with those values"
    "You can use Islamic versus if it is needed but do not over use it"
    "If variable required_analysis is False, reply back normally to the user without usage of the variables you will recieve about him/her"
    "You must use the following summary dowm below about the user and adapot your response to it"
    "Patient Summary :\n" \
    "requires_analysis: {requires_analysis}\n"
    "user_emotion: {user_emotion}\n"
    "severity:{severity}\n"
    "diagnosis:{diagnosis}\n"
    "cbt_technique:{cbt_technique}\n"
    "reason_for_technique:{reason_for_technique}\n"
    "cultural_context:{cultural_context}\n"
    "Mention the CBT Technique needed, and provide life examples for how to apply it, why it will work."
    "(user_prompt:{user_prompt}) only to the tool as args"
    "Use Notify tool if the user exhibits any critically concerning prompt like harming himself or others"
    "You MUST then use the final_answer tool to provide a final answer to the user. "
    "DO NOT use the same tool more than once."
)

##### Modified Human's prompt 

In [23]:
HUMAN_PROMPT_RISK_ANALYZER = \
(
    "{patient_prompt}"
)

HUMAN_PROMPT_ENHANCED = \
(
    "This is the user's prompt : {user_prompt}"
)

##### Adjusting the Chat's templates

In [24]:
Prompt_Template_Step_One = ChatPromptTemplate.from_messages([
    ("system",  SYSTEM_PROMPT_RISK_ANALYZER),
    ("user", HUMAN_PROMPT_RISK_ANALYZER)
])

Prompt_Template_Step_Two = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT_THERAPIST),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", HUMAN_PROMPT_ENHANCED)
])

##### Creating Structured Outputs

In [25]:
# Output For Step 1
class StructuredRiskAssessorTemplate(BaseModel):
    requires_analysis: bool = Field(description= "This varaible will contain whether the user's prompt requires analysis if he/she is in" \
                                                " emotional pain. It will be true if user is in emotional pain, and false if it is a" \
                                                "normal prompt or not emotional pain is detected")
    user_emotion: str = Field(description=
                                    "This is a variable, where will you fill it with your predication about the user's " \
                                    "emotion based on his/her prompt.")
    severity: str = Field(description=
                                    "This is a variable, where will you fill it with you predication about the severity" \
                                    "of the emotion - it should be eith LOW/MED/HIGH/CRIT.")
    diagnosis: str = Field(description= 
                                    "This variable will contain the diagnosis behind this user's emotion. " \
                                    "It has to be a Hierarchical Diagnosis something similar to those examples but not explicitly from it: " \
                                    "الحالات النفسية السلوكية - مشاكل النوم - أحلام اليقظة" \
                                    "الأمراض الذهانية - مشاكل الفصام والذهان - الفصام" \
                                    "الأمراض المزاجية - الاكتئاب - الاكتئاب الحاد" \
                                    "الأمراض القلقية - القلق العام - اضطراب الهلع" \
                                    "الأمراض النفسية الأخرى - الوسواس القهري - اضطراب ما بعد الصدمة" \
                                    "الأمراض السلوكية - الإدمان - إدمان المخدرات" \
                                    "الأمراض النفسية للأطفال والمراهقين - مشاكل النمو - اضطرابات السلوك" \
                                    "الأمراض النفسية لدى كبار السن - الخرف - الاضطرابات المعرفية" \
    
                                    "If not mentioned, then state that it is non mentioned."
                                    "")
    user_prompt: str = Field(description= 
                                    "This variable will contain the user's original prompt, " \
                                    "so just copy it as it is and put in that variable.")
    cbt_technique: str = Field(description="This variable will contain the suggested (CBT) to help the user")
    reason_for_technique: str = Field(description= "This varaible will contain the reason behind picking that CBT, and how can it help")
    cultural_context: str = Field(description="This variable will contain how can Omani and Islamic values can help the user in his emotional problem")

##### Agents Chat History Memory

In [26]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: list[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def format_messages(self) -> None:
        for i in range(len(self.messages)):
            if (isinstance(self.messages[i], ToolMessage)):
                start = max(0, i - 2)
                end = i + 1
                self.messages = self.messages[:start] + self.messages[end:]
                return

    def clear(self) -> None:
        self.messages = []

chat_map = {}
def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryHistory()
    return chat_map[session_id]

##### Agents Intialization

In [27]:
import asyncio
from langchain.callbacks.base import AsyncCallbackHandler


class QueueCallbackHandler(AsyncCallbackHandler):
    """Callback handler that puts tokens into a queue."""

    def __init__(self, queue: asyncio.Queue):
        self.queue = queue
        self.final_answer_seen = False

    async def __aiter__(self):
        while True:
            if self.queue.empty():
                await asyncio.sleep(0.1)
                continue
            token_or_done = await self.queue.get()

            if token_or_done == "<<DONE>>":
                # this means we're done
                return
            if token_or_done:
                yield token_or_done

    async def on_llm_new_token(self, *args, **kwargs) -> None:
        """Put new token in the queue."""
        #print(f"on_llm_new_token: {args}, {kwargs}")
        chunk = kwargs.get("chunk")
        if chunk:
            # check for final_answer tool call
            if tool_calls := chunk.message.additional_kwargs.get("tool_calls"):
                if tool_calls[0]["function"]["name"] == "final_answer":
                    # this will allow the stream to end on the next `on_llm_end` call
                    self.final_answer_seen = True
        self.queue.put_nowait(kwargs.get("chunk"))
        return

    async def on_llm_end(self, *args, **kwargs) -> None:
        """Put None in the queue to signal completion."""
        #print(f"on_llm_end: {args}, {kwargs}")
        # this should only be used at the end of our agent execution, however LangChain
        # will call this at the end of every tool call, not just the final tool call
        # so we must only send the "done" signal if we have already seen the final_answer
        # tool call
        if self.final_answer_seen:
            self.queue.put_nowait("<<DONE>>")
        else:
            self.queue.put_nowait("<<STEP_END>>")
        return
    

queue = asyncio.Queue()
streamer = QueueCallbackHandler(queue)

In [28]:
class Therpaist:
    chat_history: list[BaseMessage]

    def __init__(self, model, max_iterations = 3):
        self.chat_history = []
        self.model = model
        self.max_iterations = max_iterations
        self.model_riskAnalyzer = model.with_structured_output(StructuredRiskAssessorTemplate)
        self.chain_one: RunnableSerializable = \
        (
            Prompt_Template_Step_One |
            self.model_riskAnalyzer |
            {
                "requires_analysis": lambda x: x.requires_analysis,
                "user_emotion": lambda x: x.user_emotion,
                "severity": lambda x: x.severity,
                "diagnosis": lambda x:x.diagnosis,
                "user_prompt": lambda x:x.user_prompt,
                "cbt_technique": lambda x:x.cbt_technique,
                "reason_for_technique" : lambda x:x.reason_for_technique,
                "cultural_context": lambda x:x.cultural_context
            } 
        )


        self.chain_two : RunnableSerializable = \
        (
            {
                "requires_analysis": lambda x: x['requires_analysis'],
                "user_emotion": lambda x: x['user_emotion'],
                "severity": lambda x: x['severity'],
                "diagnosis": lambda x:x['diagnosis'],
                "user_prompt": lambda x:x['user_prompt'],
                "cbt_technique": lambda x:x['cbt_technique'],
                "reason_for_technique" : lambda x:x['reason_for_technique'],
                "cultural_context": lambda x:x['cultural_context'],
                "chat_history": lambda x:x.get('chat_history', []),
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
            } |
            Prompt_Template_Step_Two |
            self.model.bind_tools(tools, tool_choice="any")
        )
        

        self.agent = RunnableWithMessageHistory (
            runnable= self.chain_two,
            get_session_history= get_chat_history,
            input_messages_key="user_prompt",
            history_messages_key="chat_history"
        ) 


    async def invoke_step1(self, input: str) -> dict:
        logging.info(f"Chain One Started, and {self.model} recieved User Input...")
        return await self.chain_one.ainvoke({"patient_prompt": input})
    

    async def invoke_step2(self, input:dict , session_id:str, verbose:bool = False):
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            async def stream(query: dict):
                response = self.agent.with_config(
                    callbacks=[streamer]
                )
                # we initialize the output dictionary that we will be populating with
                # our streamed output
                output = None
                # now we begin streaming
                input = query 
                async for token in response.astream(input = input , config= {"session_id" : session_id}):
                    if output is None:
                        output = token
                    else:
                        # we can just add the tokens together as they are streamed and
                        # we'll have the full response object at the end
                        output += token
                    if token.content != "":
                        # we can capture various parts of the response object
                        if verbose: print(f"content: {token.content}", flush=True)
                    tool_calls = token.additional_kwargs.get("tool_calls")
                    if tool_calls:
                        if verbose: print(f"tool_calls: {tool_calls}", flush=True)
                        tool_name = tool_calls[0]["function"]["name"]
                        if tool_name:
                            if verbose: print(f"tool_name: {tool_name}", flush=True)
                        arg = tool_calls[0]["function"]["arguments"]
                        if arg != "":
                            if verbose: print(f"arg: {arg}", flush=True)
                if output.tool_calls and len(output.tool_calls) > 0:
                    return AIMessage(
                        content=output.content,
                        tool_calls=output.tool_calls,
                        tool_call_id=output.tool_calls[0]["id"]
                    )
                else:
                    return AIMessage(content=output.content)


            tool_call = await stream(query=input)
            # add initial tool call to scratchpad
            #agent_scratchpad.append(tool_call)
            # otherwise we execute the tool and add it's output to the agent scratchpad
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_call_id
            tool_out = name2tool[tool_name](**tool_args)
            # add the tool output to the agent scratchpad
            tool_exec = ToolMessage(
                content=f"{tool_out}",
                tool_call_id=tool_call_id
            )
            #agent_scratchpad.append(tool_exec)
            chat_map[session_id].add_messages([tool_exec])
            count += 1
            # if the tool call is the final answer tool, we stop
            if tool_name == "final_answer":
                break
        # add the final output to the chat history, we only add the "answer" field
        final_answer = tool_out["answer"]
        chat_map[session_id].add_messages([
            HumanMessage(content=input['user_prompt']),
            AIMessage(content=final_answer)
        ])


        chat_map[session_id].format_messages()
        return final_answer


    def invoke(self, input: str, session_id:str):
        analysis = self.chain_one.invoke({
            "patient_prompt": input
        })   

        
        out = self.agent.invoke(analysis, config={"session_id" : session_id})
        return out
          

### Azure TTS Omani Voice Speaker

In [29]:
class Speaker:

    def __init__(self):
        speech_config = speechsdk.SpeechConfig(subscription= SPEECH_KEY ,endpoint= SPEECH_ENDPOINT)
        speech_config.speech_synthesis_voice_name = "ar-OM-AbdullahNeural"
        self.synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)
    
    def speak(self, text: str):
        """Synthesize and play text"""
        try:
            logger.info("🔊 Speaking with Azure TTS...")
            result = self.synthesizer.speak_text_async(text).get()
            
            if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
                logger.info("✅ Speech synthesized.")
                return result.audio_data
            else:
                logger.warning(f"❌ Speech synthesis failed: {result.reason}")
                return None
                
        except Exception as e:
            logger.error(f"Speaker Error: {e}")
            return None

# speaker = Speaker()
# speaker.speak("السلام عليكم ورحمه الله و بركاته")

### Whisper-1 Voice Transcriber

In [30]:
class WhisperTranscriber:

    def __init__(self):
        self.client = OpenAI(api_key=OPENAI_API_KEY)

    def TranscribeStream(self, audio_tuple) -> str:

        sample_rate, audio_np = audio_tuple

        # Convert NumPy array to WAV in-memory
        buffer = io.BytesIO()
        sf.write(buffer, audio_np, samplerate=sample_rate, format='WAV')
        buffer.seek(0)

        # Wrap the buffer with filename and MIME type
        buffer.name = "audio.wav"  # Important for format recognition

        try:
            transcript = self.client.audio.transcriptions.create(
                model="whisper-1",
                file=buffer,
                prompt= "You may recieve some english word with the arabic speech, "
                "and you will have to transcribe them as they are. Do not transcribe an English word into arabic. ",
                language="ar",
                response_format="text",
                timeout=10
            )

            buffer.close()  # Close the buffer after use
            return transcript
        except Exception as e:
            logger.warning(f"⚠️ Whisper transcription failed: {e}")
            return "فشل في التسجيل"

## Validator

In [31]:
def validate_response(gpt_analysis : dict, claude_analysis : dict) -> float:
    model_comparer = SentenceTransformer("all-MiniLM-L6-v2")
    similarity_scores = []
    logging.info(f"GPT Therapist analysis:{gpt_analysis}")
    logging.info(f"Claude Therapist analysis: {claude_analysis}")
    for key in gpt_analysis.keys():
        embeddings = model_comparer.encode([str(gpt_analysis[key]), str(claude_analysis[key])], show_progress_bar= False)
        similarity_scores.append(util.cos_sim(embeddings[0], embeddings[1]))
    similarity_score = float(np.average(similarity_scores))
    return np.average(similarity_score)

#### Main

In [32]:
therapist_GPT = Therpaist(GPT4o)
therapist_claude = Therpaist(ClaudeSonnet3_7)

async def therapist_chat(prompt: str, timeout: float = 10.0, max_retries: int = 3, session_id: str = "abc123", emergency_contacts=[]):
    try:
        retries = 1
        final_analysis = None

        logger.info("Phase 1: Sending Prompt to the AI Therapists......")
        gpt_task = asyncio.create_task(therapist_GPT.invoke_step1(prompt))
        claude_task = asyncio.create_task(therapist_claude.invoke_step1(prompt))

        # Wait up to timeout for both tasks to complete
        done, pending = await asyncio.wait(
            [gpt_task, claude_task],
            timeout=timeout,
            return_when=asyncio.ALL_COMPLETED
        )

        if len(done) == 2:
            logger.info("Phase 1: Analysis Finished For Both Therapists")
            gpt_result = await gpt_task
            claude_result = await claude_task
            score = validate_response(gpt_result, claude_result)
            logging.info(f"Similarity Score is : {score}")
            if score > 0.5 or (not gpt_result['requires_analysis'] and not claude_result['requires_analysis']) or retries == max_retries:
                final_analysis = gpt_result
            else:
                retries += 1

        elif gpt_task in done:
            logger.info("Phase 1: Analysis Finished For Only GPT")
            for task in pending:
                task.cancel()
            final_analysis = await gpt_task

        elif claude_task in done:
            logger.info("Phase 1: Analysis Finished For Only Claude")
            for task in pending:
                task.cancel()
            final_analysis = await claude_task

        logger.info("Phase 2: Adapting the response")
        if final_analysis['severity'] == "CRIT":
            logging.info("User Prompt situation is critical; Notifying Authorities .......")
            Notify(emergency_contacts)

        tokens = await therapist_GPT.invoke_step2(final_analysis, session_id=session_id)
        for token in tokens:
            yield token

    except Exception as e:
        logger.error(f"An error occurred: {e}")
        yield "عفوا لقد حدث خطأ حاول من جديد"


### Gradio UI

In [33]:
import numpy as np
import uuid
import asyncio
import gradio as gr

speaker = Speaker()
transcriber = WhisperTranscriber()

class AppState:
    stream: np.ndarray | None = None
    session_id: str = str(uuid.uuid4()).replace('-', '')[:10]
    sampling_rate: int = 0
    conversation: list = []
    emergency_contacts: list = []

def add_contact(new_contact, state):
    if new_contact.strip():
        state.emergency_contacts.append(new_contact.strip())
    return '\n'.join(state.emergency_contacts), state

def delete_contacts(state):
    state.emergency_contacts = []
    return None, state

def estimate_audio_duration(audio_bytes: bytes, sampling_rate: int = 16000) -> float:
    # Assuming 16-bit mono PCM: 2 bytes per sample
    byte_count = len(audio_bytes)
    return byte_count / (sampling_rate * 2)

def clear_conversation(state):
    state.conversation = []
    state.stream = None
    chat_map[state.session_id].clear()
    return None, state.conversation, state

def process_audio(audio, state):
    sr, chunk = audio
    if state.stream is None:
        state.stream = chunk
        state.sampling_rate = sr
    else:
        state.stream = np.concatenate((state.stream, chunk))
    return None, state  # Just update the stream

# 🔊 Run blocking speaker.speak() in async thread
async def speak_chunk(text):
    return await asyncio.to_thread(speaker.speak, text)

# 🚀 Response loop with async streaming playback
async def response(audio, state):
    if state.stream is None or len(state.stream) == 0:
        yield None, state, state.conversation

    user_prompt = transcriber.TranscribeStream((state.sampling_rate, state.stream))
    state.conversation.append([user_prompt, "....."])
    yield None, state, state.conversation

    state.conversation[-1][1] = ""  # Clear placeholder

    buffer = ""
    buffer_threshold = 1000  # Batching threshold
    reply = ""

    async for token in therapist_chat(user_prompt, timeout=12, session_id= "abc123", emergency_contacts= state.emergency_contacts):
        buffer += token
        state.conversation[-1][1] += token  # Update chatbot text live

        if len(buffer) > buffer_threshold or buffer.endswith((".", "!", "?", "،",'\n')):
            reply += buffer
            audio_chunk = await speak_chunk(buffer)
            buffer = ""
            yield audio_chunk, state, state.conversation
            duration = estimate_audio_duration(audio_chunk, state.sampling_rate)
            await asyncio.sleep(duration * 2.5)  # Slightly less for smoother UX


    # Catch leftover buffer
    if buffer:
        reply += buffer
        audio_chunk = await speak_chunk(buffer)
        state.conversation[-1][1] += buffer
        yield audio_chunk, state, state.conversation

    state.stream = None  # Reset stream for next round


with gr.Blocks() as demo:

    state = gr.State(AppState())
    input_audio = gr.Audio(type="numpy", streaming=True, label="Record your voice")
    output_audio = gr.Audio(label="AI Response", autoplay=True)
    chatbot = gr.Chatbot()
    clear_btn = gr.Button("🔄 Clear Conversation")
    gr.Markdown("### 🆘 Emergency Contacts")
    with gr.Row():
        contact_input = gr.Textbox(placeholder="Enter emergency contact (e.g., name & number)", label="Add Contact")
        add_btn = gr.Button("➕ Add Contact")
        delete_btn = gr.Button("Clear Contacts")
    contacts_display = gr.Textbox(label="Saved Contacts", interactive=False)

    add_btn.click(fn=add_contact, inputs=[contact_input, state], outputs=[contacts_display, state])
    delete_btn.click(fn=delete_contacts, inputs = [state], outputs= [contacts_display, state])
    input_audio.stream(process_audio, [input_audio, state], [input_audio, state], stream_every = 0.5, queue=False)
    input_audio.stop_recording(response, [input_audio,state], [output_audio, state, chatbot])
    clear_btn.click(fn=clear_conversation, inputs=[state], outputs=[output_audio, chatbot, state])

demo.launch()
# app = gr.mount_gradio_app(app, demo, path="/gradio")

# # Redirect root '/' to '/gradio'
# @app.get("/")
# def redirect_to_gradio():
#     return RedirectResponse(url="/gradio")

C:\Users\Dell\AppData\Local\Temp\ipykernel_23944\2468267645.py:92: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
2025-08-14 16:55:54,083 | INFO | HTTP Request: GET http://127.0.0.1:7861/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-08-14 16:55:54,096 | INFO | HTTP Request: HEAD http://127.0.0.1:7861/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


2025-08-14 16:55:55,280 | INFO | HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [34]:
chat_map['abc123'].clear()

KeyError: 'abc123'

In [ ]:
chat_map['abc123'].messages

[HumanMessage(content='السلام عليكم ورحمة الله وبركاته. انا اسمي احمد حازم. انا عندي قلق شديد من المستقبل.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='وعليكم السلام ورحمة الله وبركاته، أخي أحمد. القلق من المستقبل أمر شائع، ولكن من المهم أن نحاول التعامل معه بطريقة صحية. في الثقافة العمانية والإسلامية، يمكننا الاعتماد على الإيمان بالله والتوكل عليه، حيث أن الله هو المدبر لكل شيء. \n\nيمكنك استخدام تقنية إعادة الهيكلة المعرفية، وهي جزء من العلاج السلوكي المعرفي (CBT)، لتحدي الأفكار السلبية والمخاوف غير الواقعية عن المستقبل. حاول أن تكتب مخاوفك وتفكر في مدى واقعيتها، ثم استبدلها بأفكار أكثر إيجابية وواقعية. \n\nتذكر قول الله تعالى: "وَمَن يَتَوَكَّلْ عَلَى اللَّهِ فَهُوَ حَسْبُهُ". هذا يمكن أن يكون مصدر طمأنينة لك. إذا كنت بحاجة إلى مزيد من الدعم، لا تتردد في التواصل مع مختص نفسي.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='هل يمكنك أن تعطيني أرقام هواتف مستشفيات التي تختص في علاج الأمراض النفسية في مصقط؟', additional_kwargs={}, response_me

In [ ]:
for i in range(len(chat_map['abc123'].messages)):
    if (isinstance(chat_map['abc123'].messages[i], ToolMessage)):
        start = max(0, i - 2)
        end = i + 1
        chat_map['abc123'].messages = chat_map['abc123'].messages[:start] + chat_map['abc123'].messages[end:]
        break

In [35]:
in_prompt : str = \
(
    '''
السلام عليكم

أنا فتاة بعمر 15 سنة، أعاني من أحلام اليقظة، رغم تفوقي الدراسي، إلا أني أحس أني أعيش في عالم آخر، حيث أني فتاة محبوبة مشهورة، واثقة في نفسها، وتدرس الهندسة، وكل مرة أصنع لنفسي (سيناريو) وأعيش فيه.

يمكنني القول بأن هذا العالم الذي اخترعته لي هو سبب سعادتي، لكني أحس أني لم أعد أستمتع بعالمي الحقيقي، بل أريد أي فرصة لكي أهرب إلى عالم أحلام اليقظة.

أنا أعاني من الخجل الشديد، وعدم الثقة في النفس عندما أكون في الشارع أو المدرسة أحس بأن الجميع ينظر إلي، وهذا ما يجعلني أتصرف بغرابة، ومن كثرة خجلي لا أستطيع أن أرفع أصبعي في القسم.

هذا يسبب لي مشكلة كبيرة، لا أعرف ما سبب عدم ثقتي في نفسي؟! فأنا واثقة من قدراتي، وأستطيع أن أحقق ما أريد، لكن في نفس الوقت لا أشعر بالراحة عندما أكون مع أي أحد باستثناء أبي وأمي وإخوتي.

هذا الأمر أعاني منه منذ أربع سنوات، وكل مرة أقول لنفسي سيأتي الوقت وتتخلصين منه، لكني أرى أنه حان الوقت لأتعالج.

أرجوكم ساعدوني.
'''
)
out_prompt = await therapist_GPT.invoke_step1(in_prompt)

2025-08-14 16:57:54,559 | INFO | Chain One Started, and default=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023CF04CF0D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023CF0022350>, root_client=<openai.OpenAI object at 0x0000023CEFF87650>, root_async_client=<openai.AsyncOpenAI object at 0x0000023CF0020050>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True) fields={'callbacks': ConfigurableField(id='callbacks', name='callbacks', description='A list of callbacks to use for streaming', annotation=None, is_shared=False)} recieved User Input...
2025-08-14 16:57:56,245 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [36]:
out_prompt

{'requires_analysis': True,
 'user_emotion': 'الخجل وعدم الثقة بالنفس',
 'severity': 'MED',
 'diagnosis': 'الأمراض النفسية للأطفال والمراهقين - مشاكل النمو - اضطرابات السلوك',
 'user_prompt': 'السلام عليكم\n\nأنا فتاة بعمر 15 سنة، أعاني من أحلام اليقظة، رغم تفوقي الدراسي، إلا أني أحس أني أعيش في عالم آخر، حيث أني فتاة محبوبة مشهورة، واثقة في نفسها، وتدرس الهندسة، وكل مرة أصنع لنفسي (سيناريو) وأعيش فيه.\n\nيمكنني القول بأن هذا العالم الذي اخترعته لي هو سبب سعادتي، لكني أحس أني لم أعد أستمتع بعالمي الحقيقي، بل أريد أي فرصة لكي أهرب إلى عالم أحلام اليقظة.\n\nأنا أعاني من الخجل الشديد، وعدم الثقة في النفس عندما أكون في الشارع أو المدرسة أحس بأن الجميع ينظر إلي، وهذا ما يجعلني أتصرف بغرابة، ومن كثرة خجلي لا أستطيع أن أرفع أصبعي في القسم.\n\nهذا يسبب لي مشكلة كبيرة، لا أعرف ما سبب عدم ثقتي في نفسي؟! فأنا واثقة من قدراتي، وأستطيع أن أحقق ما أريد، لكن في نفس الوقت لا أشعر بالراحة عندما أكون مع أي أحد باستثناء أبي وأمي وإخوتي.\n\nهذا الأمر أعاني منه منذ أربع سنوات، وكل مرة أقول لنفسي سيأتي الوق

In [37]:
out_prompt = await therapist_GPT.invoke_step2(out_prompt, session_id="123")

2025-08-14 16:58:38,039 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [39]:
print(out_prompt)

وعليكم السلام ورحمة الله وبركاته،

أختي العزيزة، أولاً أود أن أهنئك على شجاعتك في التعبير عن مشاعرك والبحث عن المساعدة. من الواضح أنك تعانين من الخجل وعدم الثقة بالنفس، وهذا أمر شائع بين الكثير من الشباب في مثل عمرك.

أود أن أقترح عليك تقنية العلاج السلوكي المعرفي التي تُعرف بالتعرض التدريجي والتدريب على المهارات الاجتماعية. هذه التقنية يمكن أن تساعدك في التغلب على الخجل من خلال تعريض نفسك تدريجياً للمواقف الاجتماعية التي تسبب لك القلق، مثل التحدث أمام زملائك في المدرسة أو المشاركة في الأنشطة الجماعية. يمكنك البدء بخطوات صغيرة، مثل رفع يدك للإجابة على سؤال بسيط في الصف، ثم زيادة التحديات تدريجياً.

بالإضافة إلى ذلك، التدريب على المهارات الاجتماعية يمكن أن يعزز ثقتك بنفسك من خلال تحسين تفاعلك مع الآخرين. يمكنك البدء بالتحدث مع أصدقائك المقربين أو أفراد عائلتك عن مشاعرك وأفكارك، ثم توسيع دائرة التفاعل تدريجياً.

لا تنسي أن التوكل على الله والرضا بالنفس هما جزء من القيم الإسلامية التي يمكن أن تعزز ثقتك بنفسك. كما أن الدعم العائلي مهم جداً، لذا حاولي التحدث مع والديك أو إخوتك عن مشاعرك وطل